In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv(
    "/Users/nicolasroever/Documents/Promotion/Debt Crisis/debt_crisis/src/debt_crisis/data/financial_data/cip_dataset_v3.csv",
)

In [ ]:
data[data["tenor"] == "10y"].sort_values(by="date", ascending=True).head(10)

In [ ]:
data["date"] = pd.to_datetime(data["date"], format="%d%b%Y")

In [ ]:
data.head()

In [ ]:
fiscal_data = pd.read_excel(
    "/Users/nicolasroever/Documents/Promotion/Debt Crisis/debt_crisis/src/debt_crisis/data/financial_data/Fiscal-space-data.xlsx",
)

In [ ]:
fiscal_data.head()

In [ ]:
import pandas as pd

# List of sheet names
sheets = [
    "ggdy",
    "pby",
    "cby",
    "fby",
    "dfggd",
    "dffb",
    "ggdma",
    "fbma",
    "fxsovsh",
    "secnres",
    "fordebtsh",
    "concggd",
    "avglife",
    "debtduey",
    "xtdebty",
    "fxdebtall",
    "prdebty",
    "pscy",
    "stdebtall",
    "stdebtres",
    "xtdebtres",
    "xtdebtrxg",
    "sovrate",
]

# List of year columns
years = [str(year) for year in range(1990, 2023)]

# Initialize an empty list to store the dataframes
dfs = []

# Loop over the sheet names
for sheet in sheets:
    # Read the sheet into a dataframe
    df = pd.read_excel(
        "/Users/nicolasroever/Documents/Promotion/Debt Crisis/debt_crisis/src/debt_crisis/data/financial_data/Fiscal-space-data.xlsx",
        sheet_name=sheet,
    )

    # Keep only the 'Country Code' column, the year columns, and add a column with the variable name
    df = df[["Country Code", *years]]
    df["Variable"] = sheet

    # Append the dataframe to the list
    dfs.append(df)

# Concatenate all dataframes in the list
data = pd.concat(dfs)

In [ ]:
data.head()

In [ ]:
# Reshape the data to long format
data = data.melt(
    id_vars=["Country Code", "Variable"],
    var_name="Year",
    value_name="Value",
)

In [ ]:
wide_data = data.pivot_table(
    index=["Country Code", "Year"],
    columns="Variable",
    values="Value",
).reset_index()

In [ ]:
wide_data.tail()

In [ ]:
# Repeat each row four times
wide_data = wide_data.loc[wide_data.index.repeat(4)].reset_index(drop=True)

# Add a 'Quarter' column
wide_data["Quarter"] = wide_data.groupby(["Country Code", "Year"]).cumcount() + 1

# Convert 'Year' and 'Quarter' to datetime
wide_data["Date"] = pd.to_datetime(
    wide_data["Year"].astype(str) + "Q" + wide_data["Quarter"].astype(str),
)

# Keep only the end of quarter dates
wide_data["Date"] = wide_data["Date"] + pd.offsets.QuarterEnd()

# Drop the 'Year' and 'Quarter' columns
wide_data = wide_data.drop(["Year", "Quarter"], axis=1)

In [ ]:
wide_data.head()